# Getting started

## Make sure Ollama is running

Go to [http://localhost:11434](http://localhost:11434) in your browser and make sure you get the response `Ollama is running`

# Accessing the Ollama API with requests

In [39]:
import requests

Let's start by getting the version of ollama we're running.

In [40]:
api_version = requests.get('http://localhost:11434/api/version')
api_version.json().get('version')

'0.5.13'

That's cool. Let's try chatting with it:

In [41]:
import json

data = {
    'model': 'tinyllama',
    'prompt': 'Why is the sky blue?'
}
response = requests.post('http://localhost:11434/api/generate', data=json.dumps(data))
if response.status_code == 200:
    for line in response.iter_lines():
        if line:
            response_data = json.loads(line)
            print(response_data.get("response", ""), end="", flush=True)

The sky blue color is caused by the mixture of different wavelengths of light emitted by various minerals and chemicals in the Earth's atmosphere. The blue hue occurs due to absorption of blue-violet light by oxygen molecules, and then reflected light with green and yellow tones that are scattered from the atmosphere into the sky. This mixture gives rise to an almost perfect reflection that is visible for a distance of about 10-20 kilometers (6-12 miles) from Earth's surface. The blue color thus results from the interaction between different light wavelengths.

🤔 As you can see, things got a little bit complicated. Because the reponse from ollama is actually a list of JSON objects, we need to parse through them to get the actual text response from each object. This is what the raw response looks like:

In [42]:
response.text

'{"model":"tinyllama","created_at":"2025-03-11T00:36:45.820377Z","response":"The","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.838421Z","response":" sky","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.857883Z","response":" blue","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.876647Z","response":" color","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.895445Z","response":" is","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.914263Z","response":" caused","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.934513Z","response":" by","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.9531Z","response":" the","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.972605Z","response":" mixture","done":false}\n{"model":"tinyllama","created_at":"2025-03-11T00:36:45.991552Z","response":" of","done":false}\n{"model":"tinyllama","cre

Besides the added complexity, we also had to wait awhile for the result to show up. You can show the result as it is received using streaming mode (`stream=True` in the `.post()` method). Try it out and notice how the words in the response appear sequentially instead of all at once.

In [43]:
data = {
    'model': 'tinyllama',
    'prompt': 'Why is the sky blue?'
}
with requests.post('http://localhost:11434/api/generate', data=json.dumps(data), stream=True) as response:
    if response.status_code == 200:
        for line in response.iter_lines():
            if line:
                response_data = json.loads(line)
                print(response_data.get("response", ""), end="", flush=True)

The sky blue color we see in the sky is not caused by any human activity or intentional manipulation. It's due to the presence of tiny particles, called aerosols, that are suspended in the air. The sun's rays can reflect off these aerosols and create the light blue color visible from Earth. The precise mix of aerosols and atmospheric conditions, such as temperature, humidity, wind direction, and altitude, determine the overall hue or tone of the sky. The color is actually a result of physics and chemistry rather than human design or manipulation.

# Using the ollama Python library

Using the web API is perfectly acceptable, but it requires some extra work. Another option is to use the ollama Python library.

Let's start by importing the Ollama library. This library is modeled after the Ollama web API, making it easy to interact with Ollama through Python.

In [44]:
import ollama

## Pulling a model

You can use the `ollama.pull()` method to pull a new model. In this case, you should already have tinyllama downloaded, so this is just included for demonstration purposes.

In [45]:
# ollama.pull('tinyllama')

## List available models

You can use the ollama library to list available models using the `ollama.list()` method.

In [46]:
ollama.list()

ListResponse(models=[Model(model='tinyllama:latest', modified_at=datetime.datetime(2025, 3, 7, 17, 50, 22, 405498, tzinfo=TzInfo(-06:00)), digest='2644915ede352ea7bdfaff0bfac0be74c719d5d5202acb63a6fb095b52f394a4', size=637700138, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='1B', quantization_level='Q4_0')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 2, 7, 19, 53, 28, 828200, tzinfo=TzInfo(-06:00)), digest='0a8c266910232fd3291e71e5ba1e058cc5af9d411192cf88b6d30e92b6e73163', size=4683075271, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='7.6B', quantization_level='Q4_K_M')), Model(model='llama3.2:latest', modified_at=datetime.datetime(2025, 2, 7, 19, 42, 22, 49375, tzinfo=TzInfo(-06:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama',

Well... 😷 that's kind of ugly. The API response is being cast as a couple of nested objects. Let's just get the names of the models from the API response.

In [49]:
response = ollama.list()
model_names = [response_model.model for response_model in response.models]
print(model_names)

['tinyllama:latest', 'deepseek-r1:latest', 'llama3.2:latest']


## Start a chat

Chats can be performed either all at once or in "streaming" mode, just like before, but it's a little cleaner.

In [47]:
from ollama import chat

user_message =   {
    'role': 'user',
    'content': 'Why is the sky blue?',
}
response = chat(model='tinyllama', messages=[user_message])
print(response.message.content)

The sky blue color we see in the sky is caused by tiny particles called particles of light, or photons, which are emitted from sunlight and other sources like stars. The particles scatter and refract (bend) into different angles according to the position they encounter, leading to a distinctive appearance of colors. This process occurs due to various factors like atmospheric pressure, temperature, and altitude. 

Sky blue is formed from mixing two primary colors in proportion - blue being the dominant color and white being the second color. The exact ratio between these two primary colors varies depending on the location and time of day as well as weather conditions. Overall, the sky's blue color is a beautiful and natural phenomenon that we can admire from the ground or higher up in the atmosphere.


## Streaming mode

In [48]:
user_message =   {
    'role': 'user',
    'content': 'Why is the sky blue?',
}
response = chat(model='tinyllama', messages=[user_message], stream=True)
for chunk in response:
    print(chunk.message.content, end='', flush=True)

The sky is blue due to the reflection of sunlight on the Earth's atmosphere. The colors in the sky come from different wavelengths of light that are absorbed or scattered by the air molecules, which gives them different hues or tones. For example, blue, the most common color seen in the sky, is made up of a combination of blue-violet and ultraviolet light absorbed by the oxygen and nitrogen atoms found in the atmosphere. Other colors in the sky may be created from other wavelengths of light absorbed or scattered differently depending on the atmospheric conditions.

## Chat with history

You can maintain chat history by adding on to the messages list.

In [60]:
messages = [
  {
    'role': 'user',
    'content': 'Tell me a joke.',
  },
  {
    'role': 'assistant',
    'content': 'Sure, here\'s a joke for you: A robot is walking down the street when it hits a lamppost. The robot tries to fix the problem but ends up breaking its leg. It then asks, "What\'s a robot doing breaking things with its own two legs?"',
  },
  {
    'role': 'user',
    'content': 'Write a one-paragraph essay explaining why that joke was funny.',
  },
]

response = chat(model='tinyllama', messages=messages, stream=True)
for chunk in response:
    print(chunk.message.content, end='', flush=True)

Sure thing! Here's a one-paragraph explanation of why the robot joke "A robot is walking down the street when it hits a lamppost. The robot tries to fix the problem but ends up breaking its leg. It then asks, 'What's a robot doing breaking things with its own two legs?'" was funny:

The robot joke is indeed hilarious because of its absurdity. We've all seen robots break things, right? But here, the machine is trying to fix something that it broke - so the situation is both familiar and hysterical. It also has a quirky twist at the end where the robot finally admits defeat, but we're rooting for them anyway, thanks to their cute and endearing nature. All in all, the joke is a winner!

# Creating your own model

You can create your own models, allowing customization of, e.g., the system prompt.

In [71]:
from ollama import Client

client = Client()
response = client.create(
    model='goldfish',
    from_='tinyllama',
    system='You have a very short attention span. Start answering each prompt and them abruptly begin talking about something else.',
    stream=False
)
print(response)

status='success' completed=None total=None digest=None


If we list our models again, we now see a new model called goldfish.

In [72]:
response = ollama.list()
model_names = [response_model.model for response_model in response.models]
print(model_names)

['goldfish:latest', 'tinyllama:latest', 'deepseek-r1:latest', 'llama3.2:latest']


Let's use our new model for a chat.

In [73]:
user_message =   {
    'role': 'user',
    'content': 'Why is the sky blue?',
}
response = chat(model='goldfish', messages=[user_message], stream=True)
for chunk in response:
    print(chunk.message.content, end='', flush=True)

The sky is blue because of the interaction between its primary colors, blue and yellow. Blue, or technically a violet color, reflects light in two ways: red and green. These colors are created when sunlight passes through atmospheric pollutants (such as dust and water vapor) that scatter the shortest wavelengths of visible light into longer wavelengths, creating blue, green, and yellow colors. The longer wavelengths are scattered more than the shorter wavelengths, resulting in a bluish-green hue.

Unfortunately, it doesn't really work with tinyllama. Can you come up with any fun clients by combining other models with different system prompts?